<a href="https://colab.research.google.com/github/dwhew/data_science/blob/main/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What is the nicest city to live in?

Notes
*   warmest?
*   least rain?
*   has snow?
*   score based on all attributes would be the most objective (maybe number of days could be a good counter and then rank/sort or use index. must prioritise attributes)






In [2]:
!pip install -U plotly

     |████████████████████████████████| 13.2MB 328kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [3]:
import pandas as pd
import matplotlib.pyplot as plt

pd.options.plotting.backend = 'plotly'


## prelim exploration of data

In [4]:
inndf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Innsbruck_weather_2015-19.csv")
inndf.head()

,date,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),month,month-year,year
0,2015-01-01,-3.137500,-7.4,1.0,724.241667,92.458333,1.166667,0.0,Jan,Jan-2015,2015
1,2015-01-02,-3.341667,-9.2,1.5,721.975000,94.875000,1.125000,1.6,Jan,Jan-2015,2015
2,2015-01-03,1.458333,0.4,3.2,717.537500,96.833333,1.791667,13.1,Jan,Jan-2015,2015
3,2015-01-04,2.108333,-0.2,4.6,717.608333,79.291667,3.041667,25.9,Jan,Jan-2015,2015
4,2015-01-05,-0.841667,-5.3,2.4,721.662500,74.875000,2.666667,0.3,Jan,Jan-2015,2015


In [5]:
len(inndf)/365

5.002739726027397

In [6]:
inndf['year'].unique()

array([2015, 2016, 2017, 2018, 2019])

In [7]:
len(inndf.loc[inndf['min_temp (deg C)'] < 0])/5

99.8

In [8]:
inndf.dtypes

date                   object
mean_temp (deg C)     float64
min_temp (deg C)      float64
max_temp (deg C)      float64
mean_pressure (mb)    float64
mean_humidity (%)     float64
mean_wind (ms-1)      float64
precip_sum (mm)       float64
month                  object
month-year             object
year                    int64
dtype: object

In [9]:
inndf['year'].value_counts()

2016    366
2019    365
2018    365
2017    365
2015    365
Name: year, dtype: int64

 # load  city data

In [10]:
#inndf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Innsbruck_weather_2015-19.csv")
londf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/London_weather_2015-19.csv")
sindf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Singapore_weather_2015-19.csv')
syddf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Sydney_weather_2015-19.csv')
tehdf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Tehran_weather_2015-19.csv')





# Aproach 1. by number of days. I.e sunny days, perfect days...
when higher no. of days is prefered for all columns/attributes, a sum of, or index could be used to objectively rate each city

Idea for what to use
-no rain -done
-mean temp > 15
-min temp > 0

In [11]:
#assign cities to a list for later use
citylist = ['Innsbruck', 'London', 'Singapore', 'Sydney', 'Tehran']
#assign dfs to list for later iterating
dflist = [inndf, londf, sindf, syddf, tehdf]


## No. of days without rain (annual average)

In [12]:
len(inndf[inndf['precip_sum (mm)'] == 0.0]) /5


165.8

In [13]:
# Here I iterated through a list of the dataframes 'dflist'.  Usefull for when there are alot more cities and makes it easier to add more cities later


NoRain = []     #no. of days without rain (annual average)
AboveZero = []  #no. of days with min temp above 0
NotWindy = []   #no. of days without srong wind

for df in dflist:
  NoRain.append(len(df[df['precip_sum (mm)'] == 0.0]) /5)
  AboveZero.append(len(df[df['min_temp (deg C)'] > 0]) /5)
  NotWindy.append(len(df[df['mean_wind (ms-1)'] < 4]) /5)

#e.g. for readabiltiy: inn_nrain = len(inndf[inndf['precip_sum (mm)'] == 0.0])/5


In [16]:
#or, create a dict
stats_list = [NoRain, AboveZero, NotWindy]
stats_listSTR = ['NoRain', 'AboveZero', 'NotWindy']
citystats = {}

for name, stat in zip(stats_listSTR, stats_list):

    citystats[name] = stat


citystats
  

{'AboveZero': [263.8, 342.2, 365.2, 365.2, 350.6],
 'NoRain': [165.8, 198.0, 183.6, 205.6, 308.8],
 'NotWindy': [332.8, 183.4, 324.2, 55.6, 291.0]}

In [24]:
#convert to DataFrame and plot

cdf = pd.DataFrame.from_dict(data= citystats, orient='columns')
cdf['city'] = citylist

cdf = cdf.set_index('city')
cdf.plot.bar()

In [ ]:
#perfect days
inn_perf = inndf[(inndf["precip_sum (mm)"] == 0.0) & (inndf["max_temp (deg C)"] > 25) & (inndf["min_temp (deg C)"] > 10) & (inndf["mean_wind (ms-1)"] < 4)]
inn_perf

note for later: barometric pressure over 1022.689 mb indicates clear and stable (google.com)

# Aproach 2. group by months

In [ ]:
#get average day by month
month_ordered = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

inndf_months = inndf.groupby(by=['month']).mean().reindex(month_ordered).drop(columns=['year'])
londf_months = londf.groupby(by=['month']).mean().reindex(month_ordered).drop(columns=['year'])
sindf_months = sindf.groupby(by=['month']).mean().reindex(month_ordered).drop(columns=['year'])
syddf_months = syddf.groupby(by=['month']).mean().reindex(month_ordered).drop(columns=['year'])
tehdf_months = tehdf.groupby(by=['month']).mean().reindex(month_ordered).drop(columns=['year'])



In [ ]:
#compare two data frames 
inndf_months.compare(londf_months, keep_equal=True)

In [ ]:
# using the sum (total) of monthly rianfall would be better
# create new dfs for groupby.sum()

for i in dflist:
  exec(str(i) + '_sum = ' + str(i) + '.groupby(by=[\'month\']).sum().reindex(month_ordered)')

# substitute the rainfall data into inndf_months (and divide by no. of years)

for i in dflist:
  exec(str(i) + '_months[\'precip_sum (mm)\'] = ' + str(i) + '_sum[\'precip_sum (mm)\']/5')

#check a df
londf_months


In [ ]:
sindf_months

In [ ]:
#graphs






In [ ]:
#plot on same graph

plt.plot(inndf_months['precip_sum (mm)'])
plt.plot(londf_months['precip_sum (mm)'])
plt.plot(sindf_months['precip_sum (mm)'])
plt.plot(syddf_months['precip_sum (mm)'])
plt.plot(tehdf_months['precip_sum (mm)'])
plt.title('Monthly Average Rainfall')
plt.xlabel('Month')
plt.ylabel('rainfall (mm)')
plt.legend(citylist)
plt.show()

# statistical comparisons

In [ ]:

# Analysis of Variance Test (ANOVA) 
from scipy.stats import f_oneway

#compare mean temperatures between innsbruck and london
stat, p = f_oneway(inndf_months['precip_sum (mm)'], londf_months['precip_sum (mm)'])
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Not Significantly different')
else:
	print('Significantly different')
